<a href="https://colab.research.google.com/github/abhishek1998s/Battery_Aging_Analysis_with_Plotly./blob/main/Battery_Aging_Analysis_with_Plotly_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install kaggle

In [2]:
   from google.colab import files
   files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abhishekkumarsing98","key":"29c918c7768663d8c05a412380371615"}'}

In [3]:
   !mkdir -p ~/.kaggle
   !cp kaggle.json ~/.kaggle/
   !chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d patrickfleith/nasa-battery-dataset

Dataset URL: https://www.kaggle.com/datasets/patrickfleith/nasa-battery-dataset
License(s): copyright-authors
 99% 225M/228M [00:01<00:00, 209MB/s]
100% 228M/228M [00:01<00:00, 205MB/s]


In [6]:
  !unzip nasa-battery-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: cleaned_dataset/data/02576.csv  
  inflating: cleaned_dataset/data/02577.csv  
  inflating: cleaned_dataset/data/02578.csv  
  inflating: cleaned_dataset/data/02579.csv  
  inflating: cleaned_dataset/data/02580.csv  
  inflating: cleaned_dataset/data/02581.csv  
  inflating: cleaned_dataset/data/02582.csv  
  inflating: cleaned_dataset/data/02583.csv  
  inflating: cleaned_dataset/data/02584.csv  
  inflating: cleaned_dataset/data/02585.csv  
  inflating: cleaned_dataset/data/02586.csv  
  inflating: cleaned_dataset/data/02587.csv  
  inflating: cleaned_dataset/data/02588.csv  
  inflating: cleaned_dataset/data/02589.csv  
  inflating: cleaned_dataset/data/02590.csv  
  inflating: cleaned_dataset/data/02591.csv  
  inflating: cleaned_dataset/data/02592.csv  
  inflating: cleaned_dataset/data/02593.csv  
  inflating: cleaned_dataset/data/02594.csv  
  inflating: cleaned_dataset/data/02595.csv  
  inflating: cleaned_dataset/

In [94]:
import pandas as pd
import numpy as np
file_path = "/content/cleaned_dataset/metadata.csv"
data = pd.read_csv(file_path)

In [95]:
print("Dataset Info:")
print(data.info())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7565 entries, 0 to 7564
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   type                 7565 non-null   object
 1   start_time           7565 non-null   object
 2   ambient_temperature  7565 non-null   int64 
 3   battery_id           7565 non-null   object
 4   test_id              7565 non-null   int64 
 5   uid                  7565 non-null   int64 
 6   filename             7565 non-null   object
 7   Capacity             2794 non-null   object
 8   Re                   1956 non-null   object
 9   Rct                  1956 non-null   object
dtypes: int64(3), object(7)
memory usage: 591.1+ KB
None


In [96]:
print("\nFirst few rows:")
data.sample(10)


First few rows:


,type,start_time,ambient_temperature,battery_id,test_id,uid,filename,Capacity,Re,Rct
45,impedance,[2.01e+03 7.00e+00 2.80e+01 1.70e+01 3.60e+01 ...,24,B0047,45,46,00046.csv,NaN,0.062410515179020035,0.21439323318771758
5845,discharge,[2.0080e+03 4.0000e+00 2.3000e+01 2.1000e+01 2...,24,B0007,109,5846,05846.csv,1.8316722583718854,NaN,NaN
6977,impedance,[2010. 9. 21. 17. 7. ...,24,B0054,169,6978,06978.csv,NaN,0.10195748343196397,0.17163226461103365
2329,charge,[2.010e+03 4.000e+00 2.200e+01 1.000e+00 4.800...,44,B0038,39,2330,02330.csv,NaN,NaN,NaN
1298,discharge,[2.0100e+03 4.0000e+00 2.5000e+01 7.0000e+00 2...,44,B0040,68,1299,01299.csv,1.7318839084573339,NaN,NaN
598,charge,[2010. 7. 28. 18. 7. ...,4,B0046,46,599,00599.csv,NaN,NaN,NaN
6213,impedance,[2008. 5. 19. 4. 16. ...,24,B0007,477,6214,06214.csv,NaN,0.057554341890008825,0.08219829115379546
5032,discharge,[2008. 5. 22. 6. 32. ...,24,B0006,528,5033,05033.csv,1.2581373688955233,NaN,NaN
5852,impedance,[2008. 4. 24. 6. 18. ...,24,B0007,116,5853,05853.csv,NaN,0.03879098770177327,0.06280987925671022
6256,discharge,[2.008e+03 5.000e+00 2.100e+01 2.100e+01 2.000...,24,B0007,520,6257,06257.csv,1.4468161332173683,NaN,NaN


In [97]:
# Check for missing values
print("\nMissing values per column:")
print(data.isnull().sum())


Missing values per column:
type                      0
start_time                0
ambient_temperature       0
battery_id                0
test_id                   0
uid                       0
filename                  0
Capacity               4771
Re                     5609
Rct                    5609
dtype: int64


In [98]:
# Step 1: Preprocess the start_time column
def parse_time(time_str):
    time_str = time_str.replace('[', '').replace(']', '').replace('...', '')
    time_parts = [float(x) for x in time_str.split()[:5]]
    return pd.Timestamp(year=int(time_parts[0]), month=int(time_parts[1]), day=int(time_parts[2]),
                        hour=int(time_parts[3]), minute=int(time_parts[4]))

data['start_time'] = data['start_time'].apply(parse_time)

In [99]:
data.sample(10)

,type,start_time,ambient_temperature,battery_id,test_id,uid,filename,Capacity,Re,Rct
4994,charge,2008-05-20 07:33:00,24,B0006,490,4995,04995.csv,NaN,NaN,NaN
2092,discharge,2009-08-01 08:40:00,24,B0034,288,2093,02093.csv,1.3387814875085953,NaN,NaN
3072,charge,2010-06-23 03:47:00,4,B0041,77,3073,03073.csv,NaN,NaN,NaN
3551,charge,2009-06-22 09:04:00,24,B0036,38,3552,03552.csv,NaN,NaN,NaN
904,charge,2010-06-21 20:10:00,4,B0043,168,905,00905.csv,NaN,NaN,NaN
2287,impedance,2009-08-25 04:57:00,24,B0034,483,2288,02288.csv,NaN,0.12848890210288166,0.15296691204250196
7241,discharge,2010-09-22 12:07:00,4,B0056,180,7242,07242.csv,1.1397253894261876,NaN,NaN
3356,charge,2010-06-16 18:26:00,4,B0044,118,3357,03357.csv,NaN,NaN,NaN
4181,impedance,2009-04-09 23:31:00,43,B0031,22,4182,04182.csv,NaN,0.04035159218032749,0.05218954533068119
3910,impedance,2009-08-18 14:22:00,24,B0036,397,3911,03911.csv,NaN,0.1051440682725413,0.1239624732439577


In [100]:

#Filter discharge data for Capacity
discharge_data = data[(data['type'] == 'discharge') & data['Capacity'].notna()][['start_time', 'Capacity']]
discharge_data.sample(10)

,start_time,Capacity
4898,2008-05-14 04:55:00,1.3893171179312644
7149,2010-09-12 03:59:00,1.1763960515773761
3046,2010-06-21 15:19:00,0.0463714545530443
3334,2010-06-10 20:13:00,1.422973037663468
1820,2009-06-20 18:55:00,1.4866702197497013
548,2010-08-17 09:52:00,1.2231273969878986
2092,2009-08-01 08:40:00,1.3387814875085953
1946,2009-07-16 18:26:00,1.4720590241846008
6747,2010-09-23 07:51:00,1.0250054893129452
4980,2008-05-19 03:27:00,1.3156485301862682


In [101]:
#Filter impedance data for Re
re_data = data[(data['type'] == 'impedance') & data['Re'].notna()][['start_time', 'Re']]

In [102]:
#Filter impedance data for Rct
rct_data = data[(data['type'] == 'impedance') & data['Rct'].notna()][['start_time', 'Rct']]

In [124]:
# Step 2: Filter impedance data for Re and Rct
impedance_data = data[(data['type'] == 'impedance') & (data['Re'].notna() | data['Rct'].notna())]

In [125]:
# Step 3: Combine Re and Rct into a single column for plotting
impedance_long = impedance_data.melt(
    id_vars=['start_time'],
    value_vars=['Re', 'Rct'],
    var_name='Parameter',
    value_name='Resistance'
)

In [126]:
# Debugging: Print the combined data
print("\nCombined Impedance Data (Re and Rct):")
print(impedance_long)


Combined Impedance Data (Re and Rct):
              start_time Parameter            Resistance
0    2010-07-21 16:53:00        Re   0.05605783343888099
1    2010-07-21 20:31:00        Re   0.05319185850921101
2    2010-07-22 17:03:00        Re   0.05963791501051059
3    2010-07-22 20:40:00        Re   0.05512505361624278
4    2010-07-23 11:35:00        Re  0.058878485312444453
...                  ...       ...                   ...
3907 2010-09-28 16:05:00       Rct   0.15760912100719615
3908 2010-09-29 08:16:00       Rct   0.14877104776241656
3909 2010-09-29 11:53:00       Rct   0.15088524149658153
3910 2010-09-30 03:59:00       Rct   0.15274663175569908
3911 2010-09-30 07:36:00       Rct   0.15489738203707232

[3912 rows x 3 columns]


In [127]:
# Display the cleaned data
print("\nDischarge Data (Capacity):")
print(discharge_data)
print("\nImpedance Data (Re):")
print(re_data)
print("\nImpedance Data (Rct):")
print(rct_data)


Discharge Data (Capacity):
              start_time            Capacity
0    2010-07-21 15:00:00  1.6743047446975208
4    2010-07-21 21:02:00  1.5243662105099023
6    2010-07-22 01:40:00  1.5080762969973425
8    2010-07-22 06:16:00  1.4835577960067696
10   2010-07-22 10:51:00  1.4671391666146525
...                  ...                 ...
7553 2010-09-29 19:50:00  1.0285269676595319
7555 2010-09-29 23:33:00  0.9816844358987022
7557 2010-09-30 03:15:00  1.0127121434171131
7561 2010-09-30 08:08:00  1.0201379996149256
7563 2010-09-30 11:50:00  0.9907591663373165

[2794 rows x 2 columns]

Impedance Data (Re):
              start_time                    Re
1    2010-07-21 16:53:00   0.05605783343888099
3    2010-07-21 20:31:00   0.05319185850921101
13   2010-07-22 17:03:00   0.05963791501051059
15   2010-07-22 20:40:00   0.05512505361624278
17   2010-07-23 11:35:00  0.058878485312444453
...                  ...                   ...
7536 2010-09-28 16:05:00   0.09747109901247247
7546 2010

In [128]:
pip install plotly


In [129]:
import plotly.express as px

In [130]:
# Plot Capacity vs Time
fig1 = px.line(merged_data, x='start_time', y='Capacity', title='Battery Capacity vs Time',
               labels={'start_time': 'Time', 'Capacity': 'Capacity (Ah)'}, markers=True)
fig1.update_traces(line=dict(width=1), marker=dict(size=1))  # Adjust line width and marker size
fig1.update_layout(
    xaxis_title="Time",
    yaxis_title="Capacity (Ah)",
    font=dict(size=14),
    xaxis=dict(showgrid=True),  # Enable gridlines for x-axis
    yaxis=dict(showgrid=True)   # Enable gridlines for y-axis
)
fig1.show()

In [131]:
# Plot Re vs Time
fig2 = px.line(merged_data, x='start_time', y='Re', title='Electrolyte Resistance (Re) vs Time',
               labels={'start_time': 'Time', 'Re': 'Re (Ohms)'}, markers=True)
fig2.update_traces(line=dict(width=2), marker=dict(size=3))
fig2.update_layout(
    xaxis_title="Time",
    yaxis_title="Re (Ohms)",
    font=dict(size=14),
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True)
)
fig2.show()

In [134]:
# Step 4: Plot Re and Rct together
fig4 = px.line(impedance_long, x='start_time', y='Resistance', color='Parameter',
              title='Re and Rct vs Time',
              labels={'start_time': 'Time', 'Resistance': 'Resistance (Ohms)', 'Parameter': 'Impedance Type'},
              markers=True)
fig4.update_traces(line=dict(width=2), marker=dict(size=8))
fig4.update_layout(
    xaxis_title="Time",
    yaxis_title="Resistance (Ohms)",
    font=dict(size=14),
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True)
)
fig4.show()